In [1]:
import keras_nlp

# Load the model architecture
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [2]:
# Path to your weights file
weights_path = "/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5"

# Load the weights
gemma_lm.load_weights(weights_path)

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [4]:
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)


In [5]:
template = """Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
{Context}

Question:
{Question}

Response:
{Answer}
"""

In [6]:
prompt = template.format(
    Context="The Great Barrier Reef is the world's largest coral reef system, located in the Coral Sea, off the coast of Queensland, Australia. It is composed of over 2,900 individual reefs and 900 islands stretching over 2,300 kilometers. The reef is known for its biodiversity, hosting countless marine species, and is a popular destination for snorkeling and diving enthusiasts. However, it faces threats from climate change, overfishing, and pollution.",
    Question="What are some of the major threats faced by the Great Barrier Reef?",
    Answer=""
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

I0000 00:00:1733058859.948731      68 service.cc:145] XLA service 0x79f807f01980 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733058859.950782      68 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1733058860.299480      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
The Great Barrier Reef is the world's largest coral reef system, located in the Coral Sea, off the coast of Queensland, Australia. It is composed of over 2,900 individual reefs and 900 islands stretching over 2,300 kilometers. The reef is known for its biodiversity, hosting countless marine species, and is a popular destination for snorkeling and diving enthusiasts. However, it faces threats from climate change, overfishing, and pollution.

Question:
What are some of the major threats faced by the Great Barrier Reef?

Response:

The major threats to the Great Barrier Reef include climate change, overfishing, and pollution, which have led to coral bleaching and habitat degradation, impacting its biodiversity and ecological balance.

"


In [7]:
prompt = template.format(
    Context="After training a machine translation model on a large dataset, the BLEU score was calculated to evaluate the quality of the model's translations compared to a set of human-generated reference translations. The model's output was compared to the reference texts using n-gram overlap, and a brevity penalty was applied to avoid overly short translations. The BLEU score was found to be 0.75, which indicates that the model's translations closely align with the reference texts.",
    Question="What is the BLEU score of the model's output as calculated during evaluation?",
    Answer=""
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
After training a machine translation model on a large dataset, the BLEU score was calculated to evaluate the quality of the model's translations compared to a set of human-generated reference translations. The model's output was compared to the reference texts using n-gram overlap, and a brevity penalty was applied to avoid overly short translations. The BLEU score was found to be 0.75, which indicates that the model's translations closely align with the reference texts.

Question:
What is the BLEU score of the model's output as calculated during evaluation?

Response:

The BLEU score was 0.75, indicating that the model's translations were reasonably accurate and aligned with the reference texts.\


In [10]:
from kaggle_secrets import UserSecretsClient

# Retrieve the Hugging Face API token from Kaggle secrets
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")


In [12]:
!pip install PyPDF2 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.9 MB/s eta 0:00:00


In [13]:
import PyPDF2
import re
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import keras_nlp

# Download the Punkt tokenizer for sentence splitting (only need to run once)
nltk.download('punkt')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text() or ""
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

# Function to exclude references section
def exclude_references(text):
    match = re.search(r'(References|Bibliography|Works Cited)', text, re.IGNORECASE)
    if match:
        text = text[:match.start()]
    return text

# Function to split text into sentences
def split_text_into_sentences(text):
    return nltk.tokenize.sent_tokenize(text)

# Load the pre-trained model and tokenizer for question generation
question_model_name = "aayeshanakarmi/T5-QG-finetuned-squad"
question_tokenizer = AutoTokenizer.from_pretrained(question_model_name, use_auth_token=hf_token)
question_model = AutoModelForSeq2SeqLM.from_pretrained(question_model_name, use_auth_token=hf_token)

# Load SentenceTransformer model for semantic similarity
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the Gemma model for answer generation
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
weights_path = "/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5"  # Update to actual path
gemma_lm.load_weights(weights_path)

# Configure the sampler for the Gemma model
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)

# Template for answer generation
answer_template = """Instruction: Generate an answer to the question using the provided context. Provide the answer only without including any additional content.

Context:
{Context}

Question:
{Question}

Response:
"""

# Function to generate questions from a group of sentences
def generate_questions_from_group(sentence_group):
    paragraph = " ".join(sentence_group)
    inputs = question_tokenizer.encode(paragraph, return_tensors="pt", truncation=True, max_length=512)
    outputs = question_model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
    question = question_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if question.strip() == "":
        question = "No valid question generated."
    
    return question

# Function to generate answers using the Gemma model
def generate_answer_with_gemma(context, question):
    prompt = answer_template.format(Context=context, Question=question)
    response = gemma_lm.generate(prompt, max_length=256)
    return response[0]  # Return the generated answer

# Function to generate unique questions and answers
def generate_unique_questions_and_answers(sentences):
    # Create embeddings for all sentences
    embeddings = sentence_model.encode(sentences)
    qa_pairs = []

    # Group sentences based on similarity and generate questions and answers
    for i in range(len(embeddings)):
        current_group = [sentences[i]]
        for j in range(i + 1, len(embeddings)):
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity > 0.7:
                current_group.append(sentences[j])
        
        # Generate question and answer if the group is large enough
        if len(current_group) > 1:
            question = generate_questions_from_group(current_group)
            context = " ".join(current_group)
            answer = generate_answer_with_gemma(context, question)
            qa_pairs.append((question, answer, context))
    
    return qa_pairs

# Example usage
pdf_path = "/kaggle/input/attention-is-all-you-need/7181-attention-is-all-you-need.pdf"

# Step 1: Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Step 2: Exclude the references section
pdf_text = exclude_references(pdf_text)

# Step 3: Split the text into sentences
sentences = split_text_into_sentences(pdf_text)

# Step 4: Generate unique questions and answers based on context relevance
qa_pairs = generate_unique_questions_and_answers(sentences)

# Step 5: Save generated questions and answers to a file or print them
output_file = "/kaggle/working/generated_questions_and_answers_with_gemma.txt"
with open(output_file, "w") as f:
    for i, (question, answer, context) in enumerate(qa_pairs):
        f.write(f"Question {i+1}: {question}\n")
        f.write(f"Answer {i+1}: {answer}\n")
        f.write(f"Context {i+1}: {context}\n\n")

print(f"{len(qa_pairs)} question-answer pairs generated and saved to '{output_file}'.")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

35 question-answer pairs generated and saved to '/kaggle/working/generated_questions_and_answers_with_gemma.txt'.
